<a href="https://colab.research.google.com/github/Rossember555/a/blob/main/Copia_de_prueba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://github.com/plotly/orca/releases/download/v1.2.1/orca-1.2.1-x86_64.AppImage -O /usr/local/bin/orca
!chmod +x /usr/local/bin/orca
!apt-get install xvfb libgtk2.0-0 libgconf-2-4

In [ ]:
import sys 
import pandas as pd
import numpy as np
from IPython.display import display, Math, Latex, Markdown,HTML
from IPython.core.display import display, HTML
from google.colab import files
from scipy.optimize import curve_fit
import plotly.graph_objects as go
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)


def data_upload(name):
    nombre = name
    uploaded = files.upload()
    for fn in uploaded.keys():
        nombre = fn
    return nombre


class data:

    """class to enter the standard data table in format "xlsx" or "csv",
    `data("url")`:enter data from a url,`data()`:enter the data from the pc.
    # solubility models
    -----------------
    `Apelblat model(data,Tf)`
    `Buchowski_ Ksiazaczak_model(data,Tf)`
    `NRTL_model(data,Tf,ΔHf)`
    `Wilson_model(data,Tf,ΔHf)`
    """
    def __init__(self,url):
        global URL
        global entorno
        self.url = url
        entorno = str(sys.executable)
        if self.url == "" and entorno == "/usr/bin/python3" :
            name = data_upload(self.url)
            URL= "/content/"+ name
        else:
            URL =self.url
       
   
    @property
    def show(self):

        L = URL.split(".")

        if L[-1]=="csv":
            df = pd.read_csv(URL)
        if L[-1]=="xlsx":
            df = pd.read_excel(URL)
        return df
    
    
    @property
    def temperature_values(self):
        df = self.show
        if "x1" in df.columns or "x2" in df.columns:
            temp = df.drop(['x1',"x2"], axis=1).columns[1:]
        else:
            temp = df.columns[1:]
        return pd.DataFrame({"T":temp})
    
    @property
    def mass_fractions(self):
        df = self.show
        fm = df["w1"]
        df_fm = pd.DataFrame({"w1":fm})
        return df_fm

    @property
    def experimental_values(self):
        df = self.show
        if "x1" in df.columns or "x2" in df.columns:
            df_ev = df.drop(['x1',"x2"], axis=1)
        else:
            df_ev = df
        return df_ev


    def molar_fractions(self,mf):
        if mf == "x1":
            df_mf = print("Does not exist mole fraction for "+ mf)
        if mf == "x2":
            df_mf = print("Does not exist mole fraction for "+ mf)
        if mf == "x3":
            df_mf = self.experimental_values
        return df_mf

#CLASE PARA EL MODELO DE SOLUBILIDAD λh

class modified_apelblat_model(data):
    def __init__(self, name):
        self.name = name

    
    @property
    def show(self):
        L = URL.split(".")

        if L[-1]=="csv":
            df = pd.read_csv(URL)
            if "x1" in df.columns or "x2" in df.columns:
                DFF = df.drop(['x1',"x2"], axis=1).rename({'w1': ''}, axis=1).set_index('').transpose().reset_index().rename({'index': 'T'}, axis=1).astype(float)
            else:
                DFF = df.rename({'w1': ''}, axis=1).set_index('').transpose().reset_index().rename({'index': 'T'}, axis=1).astype(float)
        
        if L[-1]=="xlsx":
            df = pd.read_excel(URL)
            if "x1" in df.columns or "x2" in df.columns:
                DFF = df.drop(['x1',"x2"], axis=1).rename({'w1': ''}, axis=1).set_index('').transpose().reset_index().rename({'index': 'T'}, axis=1).astype(float)
            else:
                DFF = df.rename({'w1': ''}, axis=1).set_index('').transpose().reset_index().rename({'index': 'T'}, axis=1).astype(float)

        return DFF

    @property
    def temperature_values(self):
        df = self.show
        tem = df["T"]
        return pd.DataFrame({"T":tem})
                

    @property
    def mass_fractions(self):
        df = self.show
        mf = df.columns[1:]
        return pd.DataFrame({"w1":mf})

    @property
    def equation(self):
        salida = display(HTML('<h2> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Modified Apelblat Equation</h2>'))
        display(Math(r'$$\ln(x_3) = A + \dfrac{B}{T} + C \cdot \ln T$$'))
        return salida

 
    def __kernel(self, method="lm",p0 =[1,1,1], maxfev=20000, sd = False, opt = "calculate"):
        
        def fT(T,A,B,C):
            return np.exp(A + B/T + C*np.log(T))


        df = self.show
        W  = df.columns[1:].tolist()
        Temp = df["T"].values
        

        para_A,para_B,para_C = [],[],[]
        desv_A,desv_B,desv_C = [],[],[]
        desv_para_A,desv_para_B,desv_para_C = [],[],[]
        L_para,L_desv,L_desv_para= [para_A,para_B,para_C],[desv_A,desv_B,desv_C],[desv_para_A,desv_para_B,desv_para_C]
        
        for i in  W:
            xdat = df[i]
            Tdat = df["T"]
            popt, mcov= curve_fit(fT,Tdat,xdat,method= "lm",p0=p0,maxfev=20000)
            
            for j in L_para:
                j.append(popt[L_para.index(j)])
                
            for k in L_desv:
                k.append(np.sqrt((np.diag(mcov))[L_desv.index(k)]))
                
            for l in L_desv_para:
                l.append(str(popt[L_desv_para.index(l)].round(3)) + " ± " + str(np.sqrt((np.diag(mcov))[L_desv_para.index(l)]).round(3)))

    
        C_w, C_temp, C_exp, C_cal, C_RD  = [],[],[],[],[]
      

        for i in W:

            wdat = len(Temp)*[i]
            Wdat = wdat

            tdat = Temp
            Tdat = tdat.tolist()

            xexp = df[i].values
            Xexp =  xexp.tolist()

            xcal = fT(tdat,para_A[W.index(i)],para_B[W.index(i)],para_C[W.index(i)])
            Xcal = xcal.tolist()

            RD = ((xcal - xexp)/xexp).tolist()

            C_w    += Wdat
            C_temp += Tdat
            C_exp  += Xexp
            C_cal  += Xcal
            C_RD   += RD
 
        arr_w   = np.array(C_w)
        arr_temp = np.array(C_temp)
        arr_exp = np.array(C_exp)
        arr_cal = np.array(C_cal)
        arr_RD  = np.array(C_RD )

        dataframe = pd.DataFrame({"w1":arr_w,'RD':arr_RD})

        MAPES = []

        for i in range(len(W)):

            df_mask = dataframe['w1'] == W[i]
            data_filter = dataframe[df_mask]
            MRDP = sum(data_filter["RD"])*100/len(data_filter["w1"])
            MAPES.append(MRDP)

        df_para      = pd.DataFrame({"w1":W,'A':para_A,'B':para_B,'C':para_C,"MRD%":MAPES})
        df_para_desv = pd.DataFrame({"w1":W,'A ± σ':desv_para_A,'B ± σ':desv_para_B,'C ± σ':desv_para_C,"MRD%":MAPES})
        df_cal       = pd.DataFrame({"w1":arr_w,'T': arr_temp,"x3_Exp":arr_exp,"x3_Cal":arr_cal, "RD":arr_RD })

        if opt == "calculate" and sd == False:
            df_kernel = df_cal
        if opt == "parameters" and sd == True:
            df_kernel = df_para_desv
        if opt == "parameters" and sd == False:
            df_kernel = df_para
        return  df_kernel 

    def parameters(self, method="lm",p0 =[1,1,1], maxfev=20000, sd = False):
        DF = self.__kernel(method=method,p0 =p0,maxfev=maxfev, sd = sd, opt = "parameters")
        return DF
    

    def values(self,method="lm",p0 =[1,1,1], maxfev=20000):
        DF = self.__kernel(method=method,p0=p0 , maxfev=maxfev, opt = "calculate")
        return DF

    def calculated_values(self,method="lm",p0 =[1,1,1], maxfev=20000):
        W = self.mass_fractions["w1"]
        DF = self.__kernel( method=method,p0=p0 , maxfev=maxfev, opt="calculate")
        L = []
        for i in W: 
            mask = DF['w1'] == i
            data_filter = DF[mask]
            line = data_filter.drop(["w1","x3_Exp","RD"],axis=1).rename({'T':'','x3_Cal':i}, axis=1).set_index('').transpose()
            L.append(line)

        df = pd.concat(L,axis =0).reset_index().rename({'index': 'w1'}, axis=1).rename({'T': ''},axis=1)
        return df


    def relative_deviations(self, method="lm",p0 =[1,1,1], maxfev=20000):
        
        W = self.mass_fractions["w1"]
        DF = self.__kernel( method=method,p0=p0 , maxfev=maxfev, opt="calculate")
        L = []
        for i in W: 
            mask = DF['w1'] == i
            data_filter = DF[mask]
            line = data_filter.drop(["w1","x3_Exp","x3_Cal"],axis=1).rename({'T':'','RD':i}, axis=1).set_index('').transpose()
            L.append(line)

        df = pd.concat(L,axis =0).reset_index().rename({'index': 'w1'}, axis=1).rename({'T': ''},axis=1)
    
        return df


    def statistics(self,method="lm",p0 =[1,1,1], maxfev=20000):
        DF = self.__kernel( method=method,p0=p0 , maxfev=maxfev, opt="calculate")

        MAPE = sum(abs(DF["RD"]))*100/len(DF["RD"])
        MRD = sum(DF["RD"])/len(DF["RD"])
        MRDP = sum(DF["RD"])*100/len(DF["RD"])

        ss_res = np.sum((DF["x3_Cal"] - DF["x3_Exp"])**2)
        ss_tot = np.sum((DF["x3_Exp"] - np.mean(DF["x3_Exp"]))**2)

        RMSD = np.sqrt(ss_res/len(DF["x3_Exp"]))

        k = 2  # Número de parámetros del modelo
        Q = 1   # Número de variables independientes
        N =len(DF["RD"])
        AIC = N*np.log(ss_res/N)+2*k
        AICc = abs(AIC +((2*k**2+2*k)/(N-k-1)))

        R2 = 1 - (ss_res / ss_tot)

        R2_a = 1-((N-1)/(N-Q-1))*(1- R2**2)

        L_stad= [MRDP,MAPE,RMSD,AICc,R2,R2_a]
        names = ["MRD%", "MAPE","RMSD","AICc","R2","R2_a"]

        df_estadis = pd.DataFrame({"statistic":names,"values":L_stad})

        return df_estadis

    def statistic_MRD(self,  method="lm",p0 =[1,1,1], maxfev=20000):
        MRD= self.statistics( method=method,p0 =p0, maxfev=maxfev)["values"][0]
        return print("Mean relative deviations, MRD = ",MRD)


    def statistic_MAPE(self,method="lm",p0 =[1,1,1], maxfev=20000):
        MAPE= self.statistics( method=method,p0 =p0, maxfev=maxfev)["values"][1]
        return print("Mean Absolute Percentage Error, MAPE = ",MAPE)

    def statistic_RMSD(self,method="lm",p0 =[1,1,1], maxfev=20000):
        RMSD= self.statistics(method=method,p0 =p0, maxfev=maxfev)["values"][2]
        return print("Root Mean Square Deviation, RMSD = ",RMSD)
        
    def statistic_AIC(self,method="lm",p0 =[1,1,1], maxfev=20000):
        AIC= self.statistics( method=method,p0 =p0, maxfev=maxfev)["values"][3]
        return print("Akaike Information Criterion corrected , AICc = ",AIC)

    def statistic_R2(self,method="lm", p0 =[1,1,1],maxfev=20000):
        R2= self.statistics(method=method,p0 =p0, maxfev=maxfev)["values"][4]
        return print("Coefficient of Determination, R2 =",R2)
    
    def statistic_R2a(self,method="lm", p0 =[1,1,1],maxfev=20000):
        R2_a= self.statistics(method=method,p0 =p0, maxfev=maxfev)["values"][5]
        return print("Adjusted Coefficient of Determination, R2 =",R2_a)


    def summary(self, method="lm",p0 =[1,1,1], maxfev=20000, sd = False, name = "Buchowski-Ksiazczak"):
        
        nombre       = name
        listaval     = self.values( method = method,p0 =p0, maxfev=maxfev)
        calculados   = self.calculated_values( method = method,p0 =p0, maxfev=maxfev)
        diferencias  = self.relative_deviations( method = method,p0 =p0, maxfev=maxfev) 
        parametros   = self.parameters( method = method,p0 =p0, maxfev=maxfev,sd = sd)
        estadisticos = self.statistics(method = method,p0 =p0, maxfev=maxfev)

        DATA = pd.concat([listaval ,calculados,diferencias,parametros,estadisticos], axis=1)
        
        name_archi = URL.split("/")[-1].split(".")[-2]
        
        if entorno == "/usr/bin/python3":
            url_1 = "/content/"+ nombre + "-"+ name_archi +".xlsx"
            url_3 = "/content/"+ nombre + "-"+ name_archi +".csv"
        else:
            url_1 = nombre + "-"+ name_archi +".xlsx"
            url_3 = nombre + "-"+ name_archi +".csv"

        DATA.to_excel(url_1,sheet_name=name_archi)
        DATA.to_csv(url_3)
        return DATA

    def plot(self,method="lm",p0 =[1,1,1], maxfev=20000, name = "Buchowski-Ksiazczak"):

        nombre= name

        df_values = self.values(method="lm",p0 =p0, maxfev=20000)

        name_archi ="-" + URL.split("/")[-1].split(".")[-2]
        
        if entorno == "/usr/bin/python3":
            url_2 = "/content/"+ nombre +  name_archi +".pdf"
            url_4 = "/content/"+ nombre +  name_archi +".png"
        else:
            url_2 = nombre +  name_archi +".pdf"
            url_4 = nombre +  name_archi +".png"
            
        
        W = self.mass_fractions["w1"]
        fig = go.Figure()
        numerofilas = len(self.temperature_values)
        numerocolumnas = len(W)
        L = [numerofilas*i for i in range(numerocolumnas+2)]

        X = np.linspace(min(df_values["x3_Cal"]),max(df_values["x3_Cal"]),200)

        for i in range(len(W)):
            fig.add_trace(go.Scatter(x=df_values["x3_Cal"][L[i]:L[i+1]], y=df_values["x3_Exp"][L[i]:L[i+1]],name= "w"+str(i) + "=" + str( W[i]) ,mode='markers',marker=dict(size=6,line=dict(width=0.5,color='DarkSlateGrey'))))


        fig.add_trace(go.Scatter(x=X,y=X,name="$x3^{Exp}=x3^{Cal}$"))


        fig.update_xaxes(title = "$x3^{Cal}$", title_font=dict(size=20, family='latex', color='rgb(1,21,51)'))
        fig.update_yaxes(title = "$x3^{Exp}$ ",title_font=dict(size=20,family='latex', color='rgb(1,21,51)'))
        fig.update_layout(title='',title_font=dict(size=26, family='latex', color= "rgb(1,21,51)"),width=1010, height=550)
        fig.update_layout(legend=dict(orientation="h",y=1.2,x=0.03),title_font=dict(size=40, color='rgb(1,21,51)'))
        fig.write_image(url_2)
        fig.write_image(url_4)
        return fig.show()

    def summary_download(self,name = "Modified-Apelblat",extension= "xlsx"):
        nombre = name
        name_archi = "-" + URL.split("/")[-1].split(".")[-2]
        
        if entorno == "/usr/bin/python3":
            url= "/content/"+ nombre + name_archi +"."+extension
            salida = files.download(url)
        else:
            url= nombre + name_archi +"."+extension
            salida = url
        return salida

    def plot_download(self,name ="Modified-Apelblat",extension= "pdf"):
        nombre = name
        name_archi = "-" + URL.split("/")[-1].split(".")[-2]
        if entorno == "/usr/bin/python3":
            url= "/content/"+nombre + name_archi +"."+extension
            salida = files.download(url)
        else:
            url= "/content/"+nombre + name_archi +"."+extension
            salida = url    
        return salida


#CLASE PARA EL MODELO DE SOLUBILIDAD λh

class NRTL_model(data):
    def __init__(self, name,Tf,ΔHf):
        self.name = name
        self.Tf  = Tf
        self.ΔHf = ΔHf

                
    @property
    def equation(self):
        salida = display(HTML('<h2> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; NRTL Model Equation</h2>'))
        display(Math(r'$$\Large{\ln{\gamma_3}=\frac{x_2\tau_{23}G_{23}+x_1\tau_{13}G_{13}}{x_3+x_2G_{23}+x_1G_{13}}-\frac{x_3(x_2\tau_{23}G_{23}+x_1\tau_{13}G_{13})}{(x_3+x_2G_{23}+x_1G_{13})^2}+\frac{x_2G_{32}}{x_3G_{32}+x_2+x_1G_{12}}\left(\tau_{32}-\frac{x_3\tau_{32}G_{32}+x_1\tau_{12}G_{12}}{x_3G_{32}+x_2+x_1G_{12}}\right)+\frac{x_1G_{31}}{x_3G_{31}+x_2G_{21}+x_1}\left(\tau_{31}-\frac{x_3\tau_{31}G_{31}+x_2\tau_{21}G_{21}}{x_3G_{31}+x_2G_{21}+x_1}\right)}$$'))
        return salida


    @property
    def ideal_solubilities(self):
        df  = self.show
        Tf  = self.Tf        #Temperatura de fusión
        ΔHf = self.ΔHf       #Entalpia de fusión 
        ΔSf = ΔHf/Tf         #Entropía de fusión
        R   = 0.00831433     #Constante de los gases ideales 

        Temp = df.drop(['x1',"x2"], axis=1).columns[1:]
        col_x3id = []

        for i in Temp.astype(float):
            x3=np.exp(-((ΔHf*(Tf-i))/(R*Tf*i))+((ΔSf*(Tf-i))/(R*i))-((ΔSf/R)*np.log(Tf/i)))  #solubilidad ideal
            col_x3id .append(x3)
    
        df_solid = pd.DataFrame({"T":Temp,'x3_id':col_x3id})
        return df_solid
    
    @property
    def activity_coefficients(self):
        
        df  = self.show
        Temp = self.temperature_values["T"].values
        col_x3id = self.ideal_solubilities["x3_id"].values

        cols_γ3  = []  
        namecols = []

        for i in range(len(Temp)):
            col_γ3 = col_x3id[i]/df[Temp[i]]
            cols_γ3.append(col_γ3)
            namecols.append(str(Temp[i]))

        df_ca = pd.DataFrame(dict(zip(namecols ,cols_γ3)))
        data_ca = pd.concat([df['w1'],df_ca], axis=1)
        return data_ca

 
    def __kernel(self,method="lm",p0 =[1,1,1,1,1,1], maxfev=20000, sd = False, opt = "calculate"):
        
        def fx(X,τ12,τ13,τ21,τ23,τ31,τ32):
            α = -0.3
            x1, x2, x3= X
    
            A = (x2*τ23*np.exp(α*τ23)+x1*τ13*np.exp(α*τ13))/(x3+x2*np.exp(α*τ23)+x1*np.exp(α*τ13))         
            B = x3*(x2*τ23*np.exp(α*τ23)+x1*τ13*np.exp(α*τ13))/(x3+x2*np.exp(α*τ23)+x1*np.exp(α*τ13))**2
            C = ((x2*np.exp(α*τ32))/(x3*np.exp(α*τ32)+x2+x1*np.exp(α*τ12)))*(τ32-((x3*τ32*np.exp(α*τ32)+x1*τ12*np.exp(α*τ12))/(x3*np.exp(α*τ32)+x2+x1*np.exp(α*τ12))))
            D = ((x1*np.exp(α*τ31))/(x3*np.exp(α*τ31)+x2*np.exp(α*τ21)+x1))*(τ31-((x3*τ31*np.exp(α*τ31)+x2*τ21*np.exp(α*τ21))/(x3*np.exp(α*τ31)+x2*np.exp(α*τ21)+x1)))

            return (A-B+C+D)

            
        df = self.show

        x1dat = df["x1"]
        x2dat = df["x2"]

        
        col_x3id =  self.ideal_solubilities["x3_id"].values
        cols_γ3  =  self.activity_coefficients

        Temp = self.temperature_values["T"].values
        

        para_τ12,para_τ13,para_τ21, para_τ23,para_τ31,para_τ32  = [],[],[],[],[],[]
        desv_para_τ12,desv_para_τ13, desv_para_τ21,desv_para_τ23,desv_para_τ31,desv_para_τ32  = [],[],[],[],[],[]
        L_para,L_desv_para= [ para_τ12,para_τ13,para_τ21, para_τ23,para_τ31,para_τ32 ],[desv_para_τ12,desv_para_τ13, desv_para_τ21,desv_para_τ23,desv_para_τ31,desv_para_τ32 ]

        for i in range(len(Temp)):
            x3dat = df[Temp[i]].values  
            ydat = np.log(cols_γ3[Temp[i]]).values

            popt,mcov = curve_fit(fx,(x1dat,x2dat,x3dat),ydat,p0=p0,method="lm",maxfev=600000)


            for j in L_para:
                j.append(popt[L_para.index(j)])

            for k in L_desv_para:
                k.append(str(popt[L_desv_para.index(k)].round(3)) + " ± " + str(np.sqrt((np.diag(mcov))[L_desv_para.index(k)]).round(3)))

        C_w, C_temp, C_exp, C_cal, C_RD  = [],[],[],[],[]
      

        for i in range(len(Temp)):
            ydat =  df[Temp[i]].values
            Ydat =  ydat.tolist()


            x3dat = df[Temp[i]].values

            X = np.array([x1dat,x2dat,x3dat])

            γ3_cal = fx(X,para_τ12[i],para_τ13[i],para_τ21[i],para_τ23[i],para_τ31[i],para_τ32[i])
            x3_cal = col_x3id[i]/np.exp(γ3_cal)

            Ymodel = x3_cal.tolist()

            RD = ((ydat - x3_cal)/ydat).tolist()

            C_w += df["w1"].tolist()
            C_temp += len(df["w1"])*[Temp[i]]
            C_exp  += Ydat
            C_cal  += Ymodel
            C_RD   += RD
 
        arr_w = np.array(C_w)
        arr_tem = np.array(C_temp)
        arr_exp = np.array(C_exp)
        arr_cal = np.array( C_cal)
        arr_RD  = np.array( C_RD )

        dataframe = pd.DataFrame({"T":arr_tem,'RD':arr_RD})

        MAPES = []

        for i in range(len(Temp)):
            df_mask = dataframe['T'] == Temp[i]
            data_filter = dataframe[df_mask]
            MRDP = sum(data_filter["RD"])*100/len(data_filter["T"])
            MAPES.append(MRDP)

        df_para =pd.DataFrame({"T":Temp,'τ12':para_τ12,'τ13':para_τ13,'τ21':para_τ21,'τ23':para_τ23,'τ31':para_τ31,'τ32':para_τ32,"MRD%":MAPES})
        df_para_desv = pd.DataFrame({"T":Temp,'τ12 ± σ':desv_para_τ12,'τ13 ± σ':desv_para_τ13,'τ21 ± σ':desv_para_τ21,'τ23 ± σ':desv_para_τ23,'τ31 ± σ':desv_para_τ31,'τ32 ± σ':desv_para_τ32,"MRD%":MAPES})
        df_cal  = pd.DataFrame({"w1":arr_w,'T': arr_tem,"x3_Exp":arr_exp,"x3_Cal":arr_cal, "RD":arr_RD })

        if opt == "calculate" and sd == False:
            df_kernel = df_cal
        if opt == "parameters" and sd == True:
            df_kernel = df_para_desv
        if opt == "parameters" and sd == False:
            df_kernel = df_para
        return df_kernel

    def parameters(self, method="lm",p0 =[1,1,1,1,1,1], maxfev=20000, sd = False):
        DF = self.__kernel(method=method,p0 =p0,maxfev=maxfev, sd = sd, opt = "parameters")
        return DF
    

    def values(self,method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        DF = self.__kernel(method=method,p0=p0 , maxfev=maxfev, opt = "calculate")
        return DF

    def calculated_values(self,method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        W = self.mass_fractions["w1"]
        DF = self.__kernel( method=method,p0=p0 , maxfev=maxfev, opt="calculate")
        L = []
        for i in W: 
            mask = DF['w1'] == i
            data_filter = DF[mask]
            line = data_filter.drop(["w1","x3_Exp","RD"],axis=1).rename({'T':'','x3_Cal':i}, axis=1).set_index('').transpose()
            L.append(line)

        ideal = self.ideal_solubilities.rename({'T':'',"x3_id":"ideal"}, axis=1).set_index('').transpose()

        df = pd.concat(L+[ideal],axis =0).reset_index().rename({'index': 'w1'}, axis=1).rename({'T': ''},axis=1)
        return  df


    def relative_deviations(self,method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        
        W = self.mass_fractions["w1"]
        DF = self.__kernel(method=method,p0=p0 , maxfev=maxfev, opt="calculate")
        L = []
        for i in W: 
            mask = DF['w1'] == i
            data_filter = DF[mask]
            line = data_filter.drop(["w1","x3_Exp","x3_Cal"],axis=1).rename({'T':'','RD':i}, axis=1).set_index('').transpose()
            L.append(line)

        df = pd.concat(L,axis =0).reset_index().rename({'index': 'w1'}, axis=1).rename({'T': ''},axis=1)
    
        return df


    def statistics(self,method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        DF = self.__kernel(method=method,p0=p0 , maxfev=maxfev, opt="calculate")

        MAPE = sum(abs(DF["RD"]))*100/len(DF["RD"])
        MRD = sum(DF["RD"])/len(DF["RD"])
        MRDP = sum(DF["RD"])*100/len(DF["RD"])

        ss_res = np.sum((DF["x3_Cal"] - DF["x3_Exp"])**2)
        ss_tot = np.sum((DF["x3_Exp"] - np.mean(DF["x3_Exp"]))**2)

        RMSD = np.sqrt(ss_res/len(DF["x3_Exp"]))

        k = 6  # Número de parámetros del modelo
        Q = 3   # Número de variables independientes
        N =len(DF["RD"])
        AIC = N*np.log(ss_res/N)+2*k
        AICc = abs(AIC +((2*k**2+2*k)/(N-k-1)))

        R2 = 1 - (ss_res / ss_tot)

        R2_a = 1-((N-1)/(N-Q-1))*(1- R2**2)

        L_stad= [MRDP,MAPE,RMSD,AICc,R2,R2_a]
        names = ["MRD%", "MAPE","RMSD","AICc","R2","R2_a"]

        df_estadis = pd.DataFrame({"statistic":names,"values":L_stad})

        return df_estadis

    def statistic_MRD(self,method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        MRD= self.statistics( method=method,p0 =p0, maxfev=maxfev)["values"][0]
        return print("Mean relative deviations, MRD = ",MRD)


    def statistic_MAPE(self,method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        MAPE= self.statistics( method=method,p0 =p0, maxfev=maxfev)["values"][1]
        return print("Mean Absolute Percentage Error, MAPE = ",MAPE)

    def statistic_RMSD(self, method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        RMSD= self.statistics( method=method,p0 =p0, maxfev=maxfev)["values"][2]
        return print("Root Mean Square Deviation, RMSD = ",RMSD)
        
    def statistic_AIC(self, method="lm",p0 =[1,1,1,1,1,1], maxfev=20000):
        AIC= self.statistics( method=method,p0 =p0, maxfev=maxfev)["values"][3]
        return print("Akaike Information Criterion corrected , AICc = ",AIC)

    def statistic_R2(self,method="lm", p0 =[1,1,1,1,1,1],maxfev=20000):
        R2= self.statistics(method=method,p0 =p0, maxfev=maxfev)["values"][4]
        return print("Coefficient of Determination, R2 =",R2)
    
    def statistic_R2a(self,method="lm", p0 =[1,1,1,1,1,1],maxfev=20000):
        R2_a= self.statistics(method=method,p0 =p0, maxfev=maxfev)["values"][5]
        return print("Adjusted Coefficient of Determination, R2 =",R2_a)


    def summary(self, method="lm",p0 =[1,1,1,1,1,1], maxfev=20000, sd = False, name = "NRTL"):
        
        nombre       = name
        listaval     = self.values( method = method,p0 =p0, maxfev=maxfev)
        coeficientes = self.activity_coefficients
        calculados   = self.calculated_values( method = method,p0 =p0, maxfev=maxfev)
        diferencias  = self.relative_deviations(method = method,p0 =p0, maxfev=maxfev) 
        parametros   = self.parameters( method = method,p0 =p0, maxfev=maxfev,sd = sd)
        estadisticos = self.statistics(method = method,p0 =p0, maxfev=maxfev)

        DATA = pd.concat([listaval ,coeficientes,calculados,diferencias,parametros,estadisticos], axis=1)
        
        name_archi = URL.split("/")[-1].split(".")[-2]
        
        if entorno == "/usr/bin/python3":
            url_1 = "/content/"+ nombre + "-"+ name_archi +".xlsx"
            url_3 = "/content/"+ nombre + "-"+ name_archi +".csv"
        else:
            url_1 = nombre + "-"+ name_archi +".xlsx"
            url_3 = nombre + "-"+ name_archi +".csv"

        DATA.to_excel(url_1,sheet_name=name_archi)
        DATA.to_csv(url_3)
        return DATA

    def plot(self, method="lm",p0 =[1,1,1,1,1,1], maxfev=20000, name = "NRTL"):

        nombre= name

        df_values = self.values( method="lm",p0 =p0, maxfev=20000)

        name_archi ="-" + URL.split("/")[-1].split(".")[-2]
        
        if entorno == "/usr/bin/python3":
            url_2 = "/content/"+ nombre +  name_archi +".pdf"
            url_4 = "/content/"+ nombre +  name_archi +".png"
        else:
            url_2 = nombre +  name_archi +".pdf"
            url_4 = nombre +  name_archi +".png"
            
        Temp = self.temperature_values["T"]
        
        W   = self.mass_fractions["w1"]
        fig = go.Figure()
        numerofilas =  len(W)
        numerocolumnas =  len(Temp)
        L = [numerofilas*i for i in range(numerocolumnas+2)]

        X = np.linspace(min(df_values["x3_Cal"]),max(df_values["x3_Cal"]),200)

        for i in range(len(Temp)):
            fig.add_trace(go.Scatter(x=df_values["x3_Cal"][L[i]:L[i+1]], y=df_values["x3_Exp"][L[i]:L[i+1]],name= "T"+str(i) + "=" +Temp[i] ,mode='markers',marker=dict(size=6,line=dict(width=0.5,color='DarkSlateGrey'))))


        fig.add_trace(go.Scatter(x=X,y=X,name="$x3^{Exp}=x3^{Cal}$"))

        
        
        fig.update_xaxes(title = "$x3^{Cal}$", title_font=dict(size=20, family='latex', color='rgb(1,21,51)'))
        fig.update_yaxes(title = "$x3^{Exp}$ ",title_font=dict(size=20,family='latex', color='rgb(1,21,51)'))
        fig.update_layout(title='',title_font=dict(size=26, family='latex', color= "rgb(1,21,51)"),width=1010, height=550)
        fig.update_layout(legend=dict(orientation="h",y=1.2,x=0.03),title_font=dict(size=40, color='rgb(1,21,51)'))
        fig.write_image(url_2)
        fig.write_image(url_4)
        return fig.show()

    def summary_download(self,name = "NRTL",extension= "xlsx"):
        nombre = name
        name_archi = "-" + URL.split("/")[-1].split(".")[-2]
        
        if entorno == "/usr/bin/python3":
            url= "/content/"+ nombre + name_archi +"."+extension
            salida = files.download(url)
        else:
            url= nombre + name_archi +"."+extension
            salida = url
        return salida

    def plot_download(self,name = "NRTL",extension= "pdf"):
        nombre = name
        name_archi = "-" + URL.split("/")[-1].split(".")[-2]
        if entorno == "/usr/bin/python3":
            url= "/content/"+nombre + name_archi +"."+extension
            salida = files.download(url)
        else:
            url= "/content/"+nombre + name_archi +"."+extension
            salida = url    
        return salida


class model(modified_apelblat_model ,NRTL_model):
    pass
    
     



In [ ]:
datos=data( "https://raw.githubusercontent.com/SolubilityGroup/SolubilityModels/main/progesterone-data.csv")
datos.show

,w1,278.15,283.15,288.15,293.15,298.15,303.15,308.15,313.15,318.15,x1,x2
0,0.0,0.003474,0.004699,0.006434,0.008758,0.011372,0.015176,0.019611,0.025689,0.032771,0.000,1.000000
1,0.1,0.004135,0.005635,0.007651,0.010016,0.013420,0.017438,0.022677,0.029450,0.038110,0.038,0.962179
2,0.2,0.004902,0.006558,0.008837,0.011825,0.015643,0.020472,0.026564,0.033628,0.043439,0.081,0.918745
3,0.3,0.005776,0.007875,0.010590,0.014066,0.018082,0.023678,0.030550,0.039366,0.050537,0.132,0.868347
4,0.4,0.006768,0.009291,0.012285,0.016600,0.021239,0.027654,0.035239,0.044885,0.058350,0.191,0.809164
5,0.5,0.008212,0.010839,0.014382,0.019189,0.025000,0.032454,0.040859,0.051327,0.067012,0.261,0.738681
6,0.6,0.009506,0.012840,0.017283,0.022561,0.028850,0.037721,0.047857,0.060326,0.075074,0.347,0.653319
7,0.7,0.011551,0.015361,0.020078,0.026279,0.033605,0.043093,0.054909,0.070076,0.086158,0.452,0.547810
8,0.8,0.013446,0.017629,0.023767,0.030989,0.038841,0.050664,0.064172,0.079765,0.101364,0.586,0.414069
9,0.9,0.016160,0.021631,0.027730,0.035827,0.045945,0.057556,0.074461,0.091857,0.115677,0.761,0.239013
